In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import fiona
import collections
import matplotlib.pyplot as plt
import pyproj
import rasterio
import rasterio.mask
from rasterio.plot import show
from rasterio import features
from sentinelsat import SentinelAPI, read_geojson, geojson_to_wkt
from datetime import date, timedelta
import geopandas
import os
from shapely import geometry
import numpy as np
import utilities
import pandas as pd
import shutil
import glob
pd.set_option('display.max_columns', None)
import time
import zipfile

In [10]:
# 18-21 Jul 2019
start_date = date(2021, 3, 25)
end_date = date(2021, 4, 1)

In [11]:
path = "/home/primity/terras/bulk/"
dest = "/home/primity/terras/bulk_crops/"

In [12]:
os.makedirs(path, exist_ok=True)
os.makedirs(dest, exist_ok=True)
footprint = geojson_to_wkt(read_geojson('quadrado-macao.geojson'))
for current_date in utilities.daterange(start_date, end_date -  timedelta(1)):
    os.makedirs(path, exist_ok=True)
    #get products list from this day
    api = SentinelAPI('fernandeslouro', 'copernicospw', 'https://scihub.copernicus.eu/dhus')
    products = api.query(footprint,
                         date=(current_date, current_date + timedelta(1)),
                         platformname='Sentinel-2',
                         cloudcoverpercentage=(0, 30))
    products_df = api.to_dataframe(products)
    break
    if products:
        print(f"{current_date} - {len(products)} available products")
        # download them all
        for p in list(products_df.index):
            #time.sleep(10)
            #if products_df.loc[p]['title']+'.zip' not in os.listdir(path):
            #    utilities.download_no_fail(p, path, 'fernandeslouro', 'copernicospw', 'https://scihub.copernicus.eu/dhus')
            #print(p)
            
            #with zipfile.ZipFile(os.path.join(path, products_df.loc[p]['title']+'.zip'), 'r') as zip_ref:
            #    zip_ref.extractall(path)
            
            #os.remove(os.path.join(path, products_df.loc[p]['title']+'.zip'))
            # copy jp2 to dest
            utilities.subfolders_copy(os.path.join(path, products_df.loc[p]['title']), dest)
            break
            # save cropped jp2 with same name 
            for i in os.listdir(dest):
                with rasterio.open(os.path.join(image_dir, i)) as src:
                    out_image, out_transform = rasterio.mask.mask(src, utilities.transform_shapefile(macao_shp), crop=True, nodata=10, all_touched=True)
                    out_meta = src.meta.copy() 
                with rasterio.open(os.path.join(image_dir, i), 'w', **profile) as dst:
                    dst.write(out_image.astype(rasterio.uint8), 1)
                    
            for f in glob.glob(os.path.join(path, products_df.loc[p]['title'])):
                os.remove(f)
    

In [13]:
products_df

,title,link,link_alternative,link_icon,summary,ondemand,beginposition,endposition,ingestiondate,generationdate,orbitnumber,relativeorbitnumber,vegetationpercentage,notvegetatedpercentage,waterpercentage,unclassifiedpercentage,mediumprobacloudspercentage,highprobacloudspercentage,snowicepercentage,cloudcoverpercentage,illuminationazimuthangle,illuminationzenithangle,level1cpdiidentifier,gmlfootprint,footprint,format,processingbaseline,platformname,filename,instrumentname,instrumentshortname,size,s2datatakeid,producttype,platformidentifier,orbitdirection,platformserialidentifier,processinglevel,identifier,datastripidentifier,granuleidentifier,uuid,datatakesensingstart,sensoroperationalmode,tileid,hv_order_tileid
7dab43a8-4ec0-4bd6-8185-52d61dac1e6f,S2A_MSIL2A_20210325T112111_N0214_R037_T29TPE_2...,https://scihub.copernicus.eu/dhus/odata/v1/Pro...,https://scihub.copernicus.eu/dhus/odata/v1/Pro...,https://scihub.copernicus.eu/dhus/odata/v1/Pro...,"Date: 2021-03-25T11:21:11.024Z, Instrument: MS...",false,2021-03-25 11:21:11.024,2021-03-25 11:21:11.024,2021-03-25 20:27:57.180,2021-03-25 14:08:25,30064,37,69.237977,16.015384,0.560602,0.474557,0.271154,0.017249,0.003772,13.192436,154.902211,40.844107,S2A_OPER_MSI_L1C_TL_VGS2_20210325T132755_A0300...,"<gml:Polygon srsName=""http://www.opengis.net/g...","MULTIPOLYGON (((-6.555237 39.63587147039632, -...",SAFE,02.14,Sentinel-2,S2A_MSIL2A_20210325T112111_N0214_R037_T29TPE_2...,Multi-Spectral Instrument,MSI,1.09 GB,GS2A_20210325T112111_030064_N02.14,S2MSI2A,2015-028A,DESCENDING,Sentinel-2A,Level-2A,S2A_MSIL2A_20210325T112111_N0214_R037_T29TPE_2...,S2A_OPER_MSI_L2A_DS_VGS2_20210325T140825_S2021...,S2A_OPER_MSI_L2A_TL_VGS2_20210325T140825_A0300...,7dab43a8-4ec0-4bd6-8185-52d61dac1e6f,NaT,NaN,NaN,NaN
066812ae-8a37-4ef5-ae32-2e8442423227,S2A_MSIL2A_20210325T112111_N0214_R037_T29TNE_2...,https://scihub.copernicus.eu/dhus/odata/v1/Pro...,https://scihub.copernicus.eu/dhus/odata/v1/Pro...,https://scihub.copernicus.eu/dhus/odata/v1/Pro...,"Date: 2021-03-25T11:21:11.024Z, Instrument: MS...",false,2021-03-25 11:21:11.024,2021-03-25 11:21:11.024,2021-03-25 20:27:53.460,2021-03-25 14:08:25,30064,37,51.912016,12.261984,6.774703,0.845601,4.800436,0.254103,0.009287,27.495869,153.241336,41.243794,S2A_OPER_MSI_L1C_TL_VGS2_20210325T132755_A0300...,"<gml:Polygon srsName=""http://www.opengis.net/g...","MULTIPOLYGON (((-7.7203674 39.6545573069827, -...",SAFE,02.14,Sentinel-2,S2A_MSIL2A_20210325T112111_N0214_R037_T29TNE_2...,Multi-Spectral Instrument,MSI,1.07 GB,GS2A_20210325T112111_030064_N02.14,S2MSI2A,2015-028A,DESCENDING,Sentinel-2A,Level-2A,S2A_MSIL2A_20210325T112111_N0214_R037_T29TNE_2...,S2A_OPER_MSI_L2A_DS_VGS2_20210325T140825_S2021...,S2A_OPER_MSI_L2A_TL_VGS2_20210325T140825_A0300...,066812ae-8a37-4ef5-ae32-2e8442423227,NaT,NaN,NaN,NaN
1b9265a0-36a7-4147-ada8-75a830972714,S2A_MSIL1C_20210325T112111_N0209_R037_T29TPE_2...,https://scihub.copernicus.eu/dhus/odata/v1/Pro...,https://scihub.copernicus.eu/dhus/odata/v1/Pro...,https://scihub.copernicus.eu/dhus/odata/v1/Pro...,"Date: 2021-03-25T11:21:11.024Z, Instrument: MS...",false,2021-03-25 11:21:11.024,2021-03-25 11:21:11.024,2021-03-25 18:55:40.726,2021-03-25 13:27:55,30064,37,NaN,NaN,NaN,NaN,NaN,NaN,NaN,19.260400,NaN,NaN,S2A_OPER_MSI_L1C_TL_VGS2_20210325T132755_A0300...,"<gml:Polygon srsName=""http://www.opengis.net/g...","MULTIPOLYGON (((-6.555237 39.63587147039632, -...",SAFE,02.09,Sentinel-2,S2A_MSIL1C_20210325T112111_N0209_R037_T29TPE_2...,Multi-Spectral Instrument,MSI,810.94 MB,GS2A_20210325T112111_030064_N02.09,S2MSI1C,2015-028A,DESCENDING,Sentinel-2A,Level-1C,S2A_MSIL1C_20210325T112111_N0209_R037_T29TPE_2...,S2A_OPER_MSI_L1C_DS_VGS2_20210325T132755_S2021...,S2A_OPER_MSI_L1C_TL_VGS2_20210325T132755_A0300...,1b9265a0-36a7-4147-ada8-75a830972714,2021-03-25 11:21:11.024,INS-NOBS,29TPE,TE29P


In [ ]:
terras/bulk/S2A_MSIL2A_20210325T112111_N0214_R037_T29TPE_20210325T140825.zip'
terras/bulk/S2A_MSIL2A_20210325T112111_N0214_R037_T29TPE_20210325T140825.zip